<a href="https://colab.research.google.com/github/alwansan/C-App/blob/main/mp3_to_srt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q faster-whisper ffmpeg-python
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 93.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from faster_whisper import WhisperModel
import os

AUDIO_FILE = "/content/1.mp3"

# Extract base filename (without extension)
base_name = os.path.splitext(os.path.basename(AUDIO_FILE))[0]

txt_path = f"/content/{base_name}.txt"
srt_path = f"/content/{base_name}.srt"

# Helper: CapCut-compatible SRT timestamp
def format_srt_time(seconds: float) -> str:
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{secs:02},{millis:03}"

# Load model
model = WhisperModel(
    "large-v3",
    device="cuda",
    compute_type="float16"
)

# Transcribe
segments, info = model.transcribe(
    AUDIO_FILE,
    beam_size=5,
    language='ja',
    condition_on_previous_text=False,
    vad_filter=True
)

print(f"Detected language: {info.language}")
print(f"Output files: {base_name}.txt, {base_name}.srt")

# Write outputs
with open(txt_path, "w", encoding="utf-8") as txt, \
     open(srt_path, "w", encoding="utf-8") as srt:

    for i, segment in enumerate(segments, start=1):
        text = segment.text.strip()

        txt.write(text + "\n")

        start = format_srt_time(segment.start)
        end = format_srt_time(segment.end)

        srt.write(f"{i}\n")
        srt.write(f"{start} --> {end}\n")
        srt.write(text + "\n\n")

print("✅ Transcription complete")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Detected language: ja
Output files: 1.txt, 1.srt
✅ Transcription complete


In [4]:
# ==========================================
# 1. تثبيت مكتبة الترجمة
# ==========================================
!pip install -q deep-translator

from deep_translator import GoogleTranslator
import re
from tqdm import tqdm  # شريط تقدم لمعرفة الوقت المتبقي

# ==========================================
# 2. إعدادات الملفات
# ==========================================
input_file = "1.srt"          # الملف الياباني الناتج
output_file = "1_arabic.srt"  # اسم الملف العربي الجديد

print("🚀 جاري البدء في ترجمة ملف الترجمة من اليابانية للعربية...")
print("⚠️ ملاحظة: العملية قد تستغرق بضع دقائق حسب طول الفلم...")

# دالة للتحقق هل السطر هو توقيت أم لا
def is_timestamp(line):
    return '-->' in line

# دالة للتحقق هل السطر هو رقم تسلسلي
def is_index(line):
    return line.strip().isdigit()

# إعداد المترجم
translator = GoogleTranslator(source='ja', target='ar')

translated_lines = []
text_buffer = []

# قراءة الملف الأصلي
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# المعالجة والترجمة
for line in tqdm(lines, desc="جاري الترجمة"):
    line = line.strip()

    if not line: # سطر فارغ
        translated_lines.append("\n")
        continue

    if is_index(line) or is_timestamp(line):
        translated_lines.append(line + "\n")
    else:
        # هذا سطر نصي، نقوم بترجمته
        try:
            # ترجمة النص
            translated_text = translator.translate(line)
            translated_lines.append(translated_text + "\n")
        except Exception as e:
            # في حال حدوث خطأ، نبقي النص الأصلي
            translated_lines.append(line + "\n")

# ==========================================
# 3. حفظ الملف العربي
# ==========================================
with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(translated_lines)

print(f"\n✅ تمت الترجمة بنجاح!")
print(f"📄 الملف الجديد محفوظ باسم: {output_file}")
print("يمكنك تحميله الآن من القائمة الجانبية.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.2 MB/s eta 0:00:00
🚀 جاري البدء في ترجمة ملف الترجمة من اليابانية للعربية...
⚠️ ملاحظة: العملية قد تستغرق بضع دقائق حسب طول الفلم...


جاري الترجمة: 100%|██████████| 2708/2708 [10:15<00:00,  4.40it/s]


✅ تمت الترجمة بنجاح!
📄 الملف الجديد محفوظ باسم: 1_arabic.srt
يمكنك تحميله الآن من القائمة الجانبية.
